# LangChain and Azure AI Foundry

This notebook explain how to use `langchain-azure-ai` package with the capabilities in Azure AI Foundry.

## 1. Prerequisites

To run this tutorial you need either:

1. Using GitHub Models:

    1. You can use [GitHub models](https://github.com/marketplace/models) endpoint including the free tier experience.
    2. Use the endpoint `https://models.inference.ai.azure.com` along with your GitHub Token.

1. Using Azure AI Foundry:

    1. Create an [Azure subscription](https://azure.microsoft.com).
    2. Create an Azure AI hub resource as explained at [How to create and manage an Azure AI Studio hub](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/create-azure-ai-resource).
    3. Deploy one model supporting the [Azure AI model inference API](https://aka.ms/azureai/modelinference). In this example we use a `text-embedding-3-large` deployment. 

        * You can follow the instructions at [Add and configure models to Azure AI model inference service](https://learn.microsoft.com/azure/ai-studio/ai-services/how-to/create-model-deployments).

Install the following packages:

```bash
pip install -U langchain-core langchain-azure-ai
```

## 2. Use embeddings models

Create a client to connect to the endpoint. In this case, we are working with an embeddings model hence we import the class `AzureAIEmbeddingsModel`.

In [ ]:
import os
from langchain_azure_ai.embeddings import AzureAIEmbeddingsModel

embed_model = AzureAIEmbeddingsModel(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"],
    credential=os.environ["AZURE_INFERENCE_CREDENTIAL"],
    model="text-embedding-3-large",
)

> In the previous example, we are indicating the parameter `model_name` since our endpoint has multiple models deployed on it. If your endpoint has only 1 model deployed, like with Serverless API Endpoints, you don't need to indicate the parameter `model_name`.

Now, let's create an in memory vector store to demonstrate how to use the model:

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embed_model)

Let's add some documents to the store. Adding documents will call the embeddings model:

In [ ]:
from langchain_core.documents import Document

document_1 = Document(id="1", page_content="foo", metadata={"baz": "bar"})
document_2 = Document(id="2", page_content="thud", metadata={"bar": "baz"})

documents = [document_1, document_2]
vector_store.add_documents(documents=documents)

We can then search by similarity:

In [ ]:
results = vector_store.similarity_search(query="thud", k=1)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

Or using asynchronous calling:

In [ ]:
results = await vector_store.asimilarity_search(query="thud", k=1)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")